In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

import dataset_utils
import importlib
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [3]:
DATA_DIR = "data/entire_field_raw_3_class/"
NUM_POINTS = 100
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: jumping
processing class: walking


In [4]:
def ConvBlock(x, filters, kernel_size=1, stride=1, padding='valid'):
    """Convolutional Block with Batch Normalization and ReLU activation."""
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding=padding)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

In [6]:

# Input layer for point cloud data

input_shape = (NUM_POINTS, 3)
num_classes = NUM_CLASSES
inputs = layers.Input(shape=input_shape)

# Expand dimensions for point-wise convolution
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(inputs)

# Convolutional Layers
x = ConvBlock(x, filters=64, kernel_size=(1, input_shape[1]))
x = ConvBlock(x, filters=128, kernel_size=(1, 1))
x = ConvBlock(x, filters=256, kernel_size=(1, 1))

# Global Feature Aggregation
x = layers.GlobalMaxPooling2D()(x)

# Fully connected layers for classification
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)

# Output layer with softmax for classification
outputs = layers.Dense(num_classes, activation='softmax')(x)

# Build model
model = models.Model(inputs=inputs, outputs=outputs)



In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 3)]          0         
                                                                 
 lambda_1 (Lambda)           (None, 100, 3, 1)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 1, 64)        256       
                                                                 
 batch_normalization_3 (Bat  (None, 100, 1, 64)        256       
 chNormalization)                                                
                                                                 
 re_lu_3 (ReLU)              (None, 100, 1, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 100, 1, 128)       8320      
                                                           